# 最佳化決策模式設計與應用 
# DESIGN AND APPLICATIONS OF OPTIMAL DECISION MAKING MODELS

## Project 2
### 李艾霓 / H24076095 / 國立成功大學 / 統計學系111級  

#### 2019/06/27

* **0 選題動機**
* **1 問題描述及假設**
* **2 參數設定**
    * 2.1 載入預設參數
    * 2.2 參數說明
    * 2.3 寫入參數
* **3 模型**
    * 3.1 建立模型
    * 3.2 設定變數
    * 3.3 設定限制式
    * 3.4 設定目標式
    * 3.5 最佳化
* **4 檢視結果及分析**
    * 4.1 最佳值
    * 4.2 每日人力分配
* **5 Discussion**

## 0. 選題動機
　　由於比起虛構一個題目，自己更想找實際的例子來操作，因此詢問了身邊的朋友，而其中一個曾經在生技公司工作過一段時間的朋友給了我這樣的一個問題，這正是他當時獲得工作機會的情況。我請他估計了一些參數（如公司原有業務人數、成交價值、成交率等等），大多是符合真實情況的，最終我寫成這個模型。



## 1. 問題描述及假設
A公司主要販售的商品是實驗用抗體，主要的客群是學術單位如學校，因此對於業務員的分配都是學術單位。

有鑑於新創生技公司與醫院內部創業的興起，生技公司與醫院也是潛在客戶群，但公司業務人力資源有限，故正考慮是否要再招募新的業務員，以協助新市場的開發。

公司策略部門已對下週可能可以拜訪的客戶建立聯絡清單，故已知下週在各個市場可以接洽的單位數，而公司須考量原有員工及新聘業務員能力的不同，決定如何分配人力?是否要將一些業務員轉去跑生技公司與醫院?或是直接招募新的業務員來跑?

- A公司內部已有8名業務員，且不打算解聘。
- A公司每週最多可負擔的人力成本為 100,000。
- 因需準備的內容不同，同一名業務一天內只能跑同一個市場內的業務。
- 若某業務當天跑的市場內已沒有更多可接洽單位，業務可以提早下班，但同樣不能被分配去跑其他市場。
- 若新聘僱員工，則須聘滿整週。

根據上述的假設規則，A公司想知道：針對每個市場，下個禮拜週一至周五分別需分派幾位新、舊員工去進行業務工作。

In [1]:
import pandas as pd
from gurobipy import*

## 2. 參數設定
### 2.1 載入預設參數
　　A公司依照過去的成交狀況，計算出了公司對於不同單位每筆成交的平均價值（成交平均價值），以及所有接洽終能成功完成交易的比率（原公司業務員成功率）,並評估出新聘僱業務員在未來一週對各個單位大略的成功率（新聘僱業務員成功率），如下表所示。

In [2]:
sales = pd.read_csv('sales.csv',index_col=0)
sales

,可跑單數,成交平均價值,原公司業務員成功率,新聘僱業務員成功率
市場,,,,
學術單位,317,4000,0.20,0.120
生技公司,58,18000,0.15,0.100
醫院,10,50000,0.05,0.001


而下表則顯示了新舊僱員的週薪，以及他們能力範圍內，公司每日可以分派的接洽量（每日可跑單數）。

In [3]:
sales2 = pd.read_csv('sales2.csv',index_col=0)
sales2

,週薪,每日可跑單數
原公司業務員,8750,8
新聘僱業務員,7000,4


### 2.2 參數說明
- i = 0,1 分別代表舊業務以及新業務
- j = 0,1,2 分別代表學術單位、生技公司、醫院三種不同市場
- t = 1,2,3,4,5 代表下週一至週五


- R [ i, j] 表示第i種業務對第j種市場的成功率
- Avaliable [ j] 表示下週第j種市場可接洽的總單位數
- Value [ j] 表示第j種市場中平均一筆成交的價值。
- Wage [ i] 表示第i種業務的週薪
- Num [ i] 表示第i種業務每日最多可接洽量。

### 2.3 寫入參數

In [4]:
I = range(2)
J = range(3)
T = range(1,6)

In [5]:
R = {}
for i in I:
    for j in J:
        R[i,j] = sales.iat[j,i+2]
R

{(0, 0): 0.2,
 (0, 1): 0.15,
 (0, 2): 0.05,
 (1, 0): 0.12,
 (1, 1): 0.1,
 (1, 2): 0.001}

In [6]:
Avaliable = {}
Value = {}
for j in J:
    Avaliable[j] = sales.iat[j,0]
    Value[j] = sales.iat[j,1]

In [7]:
Avaliable

{0: 317, 1: 58, 2: 10}

In [8]:
Value

{0: 4000, 1: 18000, 2: 50000}

In [9]:
Wage = {}
Num = {}
for i in I:
    Wage[i] = sales2.iat[i,0]
    Num[i] = sales2.iat[i,1]

In [10]:
Wage

{0: 8750, 1: 7000}

In [11]:
Num

{0: 8, 1: 4}

## 3. 模型
### 3.1 建立模型

In [12]:
model = Model('Sales')

Academic license - for non-commercial use only


### 3.2 設定變數
- x [ i, j, t] 表示第t天、第j種市場，要分派多少名第i種業務
- rell_num [ i, j, t] 表示表示第t天、第j種市場、第i種業務實際接洽的總數

In [13]:
x = {}
real_num = {}
for i in I:
    for j in J:
        for t in T:
            x[i,j,t] = model.addVar(vtype='I', name='x%d%d%d'%(i,j,t))
            real_num[i,j,t] = model.addVar(vtype='I', name='n%d%d%d'%(i,j,t))

In [14]:
# 順便算出一週總薪資以便限制式使用
Total_wage = 8*Wage[0]+quicksum(x[1,j,1] for j in J)*Wage[1]

In [15]:
model.update()

### 3.3 設定限制式

In [16]:
# 舊有業務固定8名
for t in T:
    model.addConstr(quicksum(x[0,j,t] for j in J) == 8)

In [17]:
# 新聘業務需聘整周(人數固定)
for t in T:
    model.addConstr(quicksum(x[1,j,t] for j in J)-quicksum(x[1,j,1] for j in J) == 0)

In [18]:
# 一週最大人力成本為十萬元
model.addConstr(Total_wage <= 100000)

<gurobi.Constr *Awaiting Model Update*>

In [19]:
# 每種市場實際接洽的總量 <= 跑該市場業務人數 * 最大接洽量
for i,j,t in real_num:
    model.addConstr(real_num[i,j,t]-x[i,j,t]*Num[i] <= 0)

In [20]:
# 每個市場有最多可接洽數
for j in J:
    n = 0
    for i in I:
        for t in T:
            n += real_num[i,j,t]
    model.addConstr(n <= Avaliable[j])

### 3.4 設定目標式
目標為最大化下週獲利（假設其他成本為固定常數，可忽略不影響模型）

In [21]:
earn = quicksum(real_num[i,j,t]*R[i,j]*Value[j] for (i,j,t) in x)

In [22]:
cost = Total_wage

In [23]:
profit = earn - cost
model.setObjective(profit,GRB.MAXIMIZE)

### 3.5 最佳化

In [24]:
model.optimize()

Optimize a model with 44 rows, 60 columns and 132 nonzeros
Variable types: 0 continuous, 60 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [5e+01, 7e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 3e+04]
Found heuristic solution: objective 6200.0000000
Presolve removed 10 rows and 9 columns
Presolve time: 0.00s
Presolved: 34 rows, 51 columns, 114 nonzeros
Variable types: 0 continuous, 51 integer (0 binary)

Root relaxation: objective 3.216500e+05, 26 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 321650.000    0    3 6200.00000 321650.000  5088%     -    0s
H    0     0                    314680.00000 321650.000  2.21%     -    0s
H    0     0                    315480.00000 321650.000  1.96%     -    0s
     0     0 321000.000    0    2 315480.000 321000.000  1.75%     -    0s
 

## 4. 檢視結果及分析
### 4.1 最佳值
減去其他成本即為當週獲利

In [25]:
model.ObjVal

315480.0

### 4.2 每日人力分配

In [29]:
result = {}
for i in I:
    ls = []
    for j in J:
        sub = []
        for t in T:
            sub.append(int(x[i,j,t].X))
        ls.append(sub)
    result[i] = ls
df0 = pd.DataFrame(result[0], index=sales.index, columns=['Mon','Tue','Wen','Thu','Fri'])
df1 = pd.DataFrame(result[1], index=sales.index, columns=df0.columns)

#### 舊有業務員一週分配

In [30]:
df0

,Mon,Tue,Wen,Thu,Fri
市場,,,,,
學術單位,8,7,1,8,8
生技公司,0,0,7,0,0
醫院,0,1,0,0,0


#### 新聘僱業務員一週分配

In [31]:
df1

,Mon,Tue,Wen,Thu,Fri
市場,,,,,
學術單位,2,3,3,3,3
生技公司,1,0,0,0,0
醫院,0,0,0,0,0


## 5. Discussion
　　舊有業務員對生技公司的獲利最高，但因有剩餘零星的可接洽案件，比起讓舊有業務花一整天跑，派一位新聘僱業務處理完剩餘案件更具有比較利益，因此仍有一位新業務在其中一天被派去生技公司的市場。
  
　　醫院因其特性而有非常高的平均獲利，但成交率很低（尤其是對於新業務員），雖然還剩下兩個案件，但並不值得再多分派一位員工。
  
　　而學術單位的期望獲利雖然較另外兩者低，但仍高於聘請新業務員的薪資，因此仍驅使A公司多聘請了3位業務員來增加業務量，並且，尚未到達A公司人力成本的上限（上限是新聘請4位新業務）。

　　A公司最終決定將部分舊業務員分配到新市場，而多聘僱了3位新業務員來彌補原本在學術單位的空缺，可見新市場確實有其開發價值，且由舊業務來開發較為合適，而原本在學術單位的市場即使分配給新的員工也是對公司有益的。

　　回到現實生活中，給出這個問題的那位朋友表示，當時A公司只招募了他一位新員工，但後來也持續在面試新人，或許確實是還處於缺人狀態沒錯，只不過一直沒有找到合適人選，若有適合的求職者，是有可能再增加新員工的，與模型的結果還算是相符。